# Fonte de Dados
https://www.kaggle.com/unsdsn/world-happiness/version/1#2016.csv

O conjunto de dados contém pontuações e classificações de felicidade, que usam dados da Pesquisa Mundial Gallup. As pontuações são baseadas nas principais perguntas sobre avaliações sobre a vida feitas na pesquisa. As perguntas são baseadas na escada Cantril, que pede aos entrevistados que pensem em uma escala com a melhor vida possível. Isso pede que os entrevistados classifiquem sua vida de zero a dez. Dez é a melhor vida possível e zero é o pior. Essas pontuações foram coletadas de 2013 a 2016, o que torna esses dados ainda muito relevantes.

As colunas deste conjunto de dados incluem país, região, rank de felicidade, pontuação de felicidade, intervalo de confiança inferior, intervalo de confiança superior, Economia (PIB), Família, Saúde (Expectativa de Vida), Liberdade, Confiança no Governo, Generosidade, Distopia residual. As colunas depois do índice de felicidade levam em consideração seis fatores que tornam as avaliações de vida mais altas em cada país. Essas seis variáveis são produção econômica, apoio social, expectativa de vida, liberdade, ausência de corrupção e generosidade. A distopia não afeta a pontuação de felicidade de um país, ela apenas possui valores iguais à menor média nacional do mundo para cada um dos seis fatores, e serve apenas como referência para comparação.

Nosso conjunto de dados contém 157 instâncias e 13 variáveis, as quais são:
  > **País** - variável nominal e contém 156 países;<br>
  > **Região** - também nominal e divide os dados em regiões de acordo com o continente;<br>
  > **Rank de felicidade** - valor numérico e classifica a felicidade de cada país de 1 a 156, com base na pontuação mundial de felicidade;<br> 
  > **Pontuação de felicidade** - valor numérico em uma escala de um a dez;<br>
  > **Nível de confiança inferior e superior** - usados para prever que ambos são numéricos;<br>
  > **Economia** - variável numérica do PIB per capita, que nos dá uma visão do desempenho econômico de um país;<br>
  > **Família** - variável numérica que nos dá uma ideia do quanto um país valoriza a família;<br>
  > **Expectativa de Vida** - os cuidados de saúde são uma variável numérica baseada no acesso e na qualidade dos cuidados de saúde;<br>
  > **Liberdade** - variável numérica que se baseia em quão livres as pessoas se sentem em um determinado país;<br>
  > **Confiança no governo** - variável numérica que se baseia em quanto as pessoas confiam no governo;<br>
  > **Generosidade** - variável numérica baseada em quão generosas as pessoas são em um determinado país;<br>
  > **Distopia Residual** - variável numérica num país imaginário que tem as pessoas menos felizes do mundo. É um país imaginário que tem as pessoas menos felizes do mundo. Essa variável é usada para comparar com outros países e avaliar seu nível de felicidade.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Survey;

CREATE TABLE Survey(
    country VARCHAR(50) NOT NULL,
    region VARCHAR(50),
    hap_rank INT NOT NULL,
    hap_score FLOAT,
    lower_confidence_interval FLOAT,
    upper_confidence_interval FLOAT,
    economy FLOAT,
    family FLOAT,
    life_expectancy FLOAT,
    freedom FLOAT,
    gov_corruption FLOAT,
    generosity FLOAT,
    dystopia_residual FLOAT,
    PRIMARY KEY(country)
) AS SELECT 
    Country,
    Region,
    Happiness_Rank,
    Happiness_Score,
    Lower_Confidence_Interval,
    Upper_Confidence_Interval,
    Economy,
    Family,
    Life_Expectancy,
    Freedom,
    Government_Corruption,
    Generosity,
    Dystopia_Residual
FROM CSVREAD('./world_happiness_report_2016.csv');

SELECT * FROM Survey;

## Análises em SQL

### Query 1
- Média do happiness score dos países, agrupados por região.

In [3]:
SELECT S.region, AVG(hap_score) avg_hap_score, COUNT(*) countries
    FROM Survey S
    GROUP BY S.region
    ORDER BY avg_hap_score DESC;

### Query 2

- Coeficientes de correlação de Pearson entre a pontuação de felicidade e os seis fatores: Economia, Família, Expectativa de Vida, Liberdade, Confiança no Governo, Generosidade. Essa consulta nos proporciona uma noção de quais dos seis fatores mais influenciam para ter uma pontuação de felicidade mais elevada.

In [4]:
DROP TABLE IF EXISTS CorrelationHapScore;

CREATE TABLE CorrelationHapScore(
    relation VARCHAR(50),
    coeficient FLOAT
);

In [5]:
DROP VIEW IF EXISTS SumHapEcon;
DROP VIEW IF EXISTS HapEcon;

CREATE VIEW HapEcon AS
SELECT S.country, S.hap_score X, S.economy Y, S.hap_score*S.economy XY, S.hap_score*S.hap_score X2, S.economy*S.economy Y2
    FROM Survey S;
    
CREATE VIEW SumHapEcon AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapEcon HE;

INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                FROM SumHapEcon SHE), 'economy');

In [6]:
DROP VIEW IF EXISTS SumHapFam;
DROP VIEW IF EXISTS HapFam;

CREATE VIEW HapFam AS
SELECT S.country, S.hap_score X, S.family Y, S.hap_score*S.family XY, S.hap_score*S.hap_score X2, S.family*S.family Y2
    FROM Survey S;
    
CREATE VIEW SumHapFam AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapFam HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                    FROM SumHapFam SHE), 'family');

In [7]:
DROP VIEW IF EXISTS SumHapLife;
DROP VIEW IF EXISTS HapLife;

CREATE VIEW HapLife AS
SELECT S.country, S.hap_score X, S.life_expectancy Y, S.hap_score*S.life_expectancy XY, S.hap_score*S.hap_score X2, S.life_expectancy*S.life_expectancy Y2
    FROM Survey S;
    
CREATE VIEW SumHapLife AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapLife HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                            FROM SumHapLife SHE), 'life_expectancy');

In [8]:
DROP VIEW IF EXISTS SumHapFree;
DROP VIEW IF EXISTS HapFree;

CREATE VIEW HapFree AS
SELECT S.country, S.hap_score X, S.freedom Y, S.hap_score*S.freedom XY, S.hap_score*S.hap_score X2, S.freedom*S.freedom Y2
    FROM Survey S;
    
CREATE VIEW SumHapFree AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapFree HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                    FROM SumHapFree SHE), 'freedom');

In [9]:
DROP VIEW IF EXISTS SumHapGov;
DROP VIEW IF EXISTS HapGov;

CREATE VIEW HapGov AS
SELECT S.country, S.hap_score X, S.gov_corruption Y, S.hap_score*S.gov_corruption XY, S.hap_score*S.hap_score X2, S.gov_corruption*S.gov_corruption Y2
    FROM Survey S;
    
CREATE VIEW SumHapGov AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapGov HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                            FROM SumHapGov SHE), 'gov_corruption');

In [10]:
DROP VIEW IF EXISTS SumHapGen;
DROP VIEW IF EXISTS HapGen;

CREATE VIEW HapGen AS
SELECT S.country, S.hap_score X, S.generosity Y, S.hap_score*S.generosity XY, S.hap_score*S.hap_score X2, S.generosity*S.generosity Y2
    FROM Survey S;
    
CREATE VIEW SumHapGen AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapGen HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                        FROM SumHapGen SHE), 'generosity');

In [12]:
SELECT * FROM CorrelationHapScore ORDER BY coeficient DESC;